## Container Objects

*Container Objects* adalah obyek yang mempunyai sifat sebagai penampung, berisi lebih dari satu obyek. Untuk memahami *container objects*, perlu dipahami gambaran umum arsitektur dari spaCy sebagai berikut:

![Arsitektur spaCy](spacy-arch.jpg)

Sumber bacaan: https://spacy.io/api

### Container Objects 1 - **Language**

Pertama kita harus pahami dulu tentang **Language**, sila baca (https://spacy.io/api/language).

In [1]:
# Definisi language - menggunakan spacy.load
import spacy
import itertools

nlp = spacy.load("en_core_web_sm")
# --- sampai di sini

print(nlp)

print(nlp.vocab.strings)

print(len(nlp.vocab.strings))
words = set(nlp.vocab.strings)
word = 'speech'
print(f"Is '{word}' an English word?: {word in words}")
word = 'mengapa'
print(f"Is '{word}' an English word?: {word in words}")

word = 'tanya' #Tanya dianggap sebagai nama dalam bahasa inggris 
print(f"Is '{word}' an English word?: {word in words}")

for i, val in enumerate(itertools.islice(words, 10)):
    print(i, val)

83839
Is 'speech' an English word?: True
Is 'mengapa' an English word?: False
Is 'tanya' an English word?: True
0 10:25
1 Shana
2 slopes
3 Stamford
4 Wilsonian
5 woodbridge
6 richest
7 nickeled
8 antiwar
9 Fireside


In [2]:
# Definisi language - cara lain
import spacy

from spacy.lang.en import English
nlp = English()
print(nlp)

In [3]:
# Jika ingin menggunakan API untuk language - vocab
# tanpa definisi bahasa tertentu
# (membangun dari awal)

from spacy.vocab import Vocab
from spacy.language import Language
nlp = Language(Vocab())
print(nlp)

**Penting**: kelas *Language* akan memproses teks dan menghasilkan obyek **Doc**. **Doc** merupakan *container objects*. Kita akan mempelajari sedikit demi sedikit dari arsitektur tersebut, dimulai dari *container objects*.

### Container Objects 2 - **Doc**

Sumber bacaan: https://spacy.io/api/doc

Merupakan *container* untuk mengakses anotasi linguistik.

In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")
# doc = nlp("A Doc is a sequence of Token")
doc = nlp("A Doc is a sequence of Token objects")

for token in doc:
    print(token)

A
Doc
is
a
sequence
of
Token
objects


In [7]:
# Inisialisasi Doc - cara lain
from spacy.tokens import Doc

words = ["hello", "world", "!"]
spaces = [True, False, False]
#spaces = [False, True, False]
doc = Doc(nlp.vocab, words=words, spaces=spaces)

for token in doc:
    print(token, len(token))
    
print(doc)

hello 5
world 5
! 1
hello world!


In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("A Doc is a sequence of Token objects")

print(doc[0].text)
print(doc[-2].text)
span = doc[1:6]
print(span.text)

A
Token
Doc is a sequence of


In [9]:
print([t.text for t in doc])

['A', 'Doc', 'is', 'a', 'sequence', 'of', 'Token', 'objects']


In [10]:
print(len(doc))

8


In [11]:
# set_extension

from spacy.tokens import Doc

city_getter = lambda doc: any(city in doc.text for city in ("Ternate", "Paris", "Berlin"))
Doc.set_extension("has_city", getter=city_getter, force=True)
doc = nlp("I like New York")
print(doc._.has_city)
extension = Doc.get_extension("has_city")
print(extension)

False
(None, None, <function <lambda> at 0x0000027C1967CE50>, None)


In [12]:
doc = nlp("I like New York")
span = doc.char_span(7, 15, label="GPE")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

New York 7 15 GPE


In [13]:
# set entity and its label
from spacy.tokens import Span

doc = nlp("Mr. Best flew to New York on Saturday morning.")

doc.set_ents([Span(doc, 0, 2, "PERSON")])
#doc.set_ents([Span(doc, 0, 2, "CITY")])
ents = list(doc.ents)

for word in ents:
    print(word.text, word.label_)

Mr. Best PERSON


In [14]:
# Doc similarity
apples = nlp("I like apples")
oranges = nlp("I like oranges")

apples_oranges = apples.similarity(oranges)
oranges_apples = oranges.similarity(apples)

print(apples_oranges)
print(oranges_apples)

0.8840670285788055
0.8840670285788055


C:\Users\EVANGS~1\AppData\Local\Temp/ipykernel_7220/3927645350.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  apples_oranges = apples.similarity(oranges)
C:\Users\EVANGS~1\AppData\Local\Temp/ipykernel_7220/3927645350.py:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive ten

### Container Objects 3 - **DocBin**

Sumber bacaan: https://spacy.io/api/docbin

Digunakan untuk *packing* obyek Doc untuk serialisasi biner. Serialisasi yang digunakan adalah [msgpack](https://msgpack.org/index.html) yang dikompres menggunakan gzip.

In [5]:
from spacy.tokens import DocBin

doc_bin = DocBin(attrs=["LEMMA"])
#doc = nlp("The DocBin class lets you efficiently serialize the information from a collection of Doc objects.")
doc = nlp("Coba iseng cetak hasil kalimat ini ya...")
doc_bin.add(doc)
for token in doc:
    print(token)  
doc_bin.to_disk("./data.spacy")

doc_bin = DocBin().from_disk("./data.spacy")
for token in doc:
    print(token)

Coba
iseng
cetak
hasil
kalimat
ini
ya
...
Coba
iseng
cetak
hasil
kalimat
ini
ya
...


File yang dihasilkan:

```
$ ls -la
total 64
...
...
-rw-r--r--  1 bpdp bpdp   602 Jan 19 18:08 data.spacy
...
...
$```

### Container Objects 4 - **Lexeme**

Sumber bacaan: https://spacy.io/api/lexeme

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I love Coffee")
for word in doc:
    lexeme = doc.vocab[word.text]
    print(lexeme.text, lexeme.orth, lexeme.shape_, lexeme.prefix_, lexeme.suffix_,
            lexeme.is_alpha, lexeme.is_digit, lexeme.is_title, lexeme.lang_)
    
# orth => orthographic features, representasi text dalam hash
# shape_ => bentuk teks, karakter => X atau x, hanya 4 awal. Jika numerik: d

I 4690420944186131903 X I I True False True en
love 3702023516439754181 xxxx l ove True False False en
Coffee 3474706295102377020 Xxxxx C fee True False True en


### Container Objects Lain

Masih ada beberapa container objects:
1. Example: merupakan koleksi dari anotasi pelatihan. https://spacy.io/api/example
2. Span: potongan dari obyek Doc. https://spacy.io/api/span
3. SpanGroup: sekumpulan dari span, bisa diberi nama: https://spacy.io/api/spangroup
4. Token: https://spacy.io/api/token